In [4]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
from IPython.display import Image
import time



def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()

    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
      
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
         
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return (locs, preds)


prototxtPath = 'deploy.prototxt'
weightsPath = 'res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("mask-detector-model.model")

vs = VideoStream(src=0).start()
time.sleep(2.0)
from PIL import Image as _Image

PhotoCntr = 0
countImg = 0;
timing = time.time()
while True:
    if time.time() - timing > 1.0:
        timing = time.time()
        print(countImg)
        countImg = 0
    frame = vs.read()
    countImg +=1
    frame = imutils.resize(frame, width=400)
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):

        (startX, startY, endX, endY) = box
        (CMFD,IMFD,NMFD) = pred

        if CMFD > IMFD and CMFD > NMFD:
            label = "CMFD"
            color = (0, 255, 0) 
        elif NMFD > IMFD and NMFD > CMFD:
            label = "NMFD"
            color = (255, 0, 0) 
        elif IMFD > CMFD and IMFD > NMFD:
            label = "IMFD"
            color = (0, 0, 255) 
        label = "{}: {:.2f}%".format(label, max(CMFD,IMFD,NMFD) * 100)
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q') or key == ord('й'):
        break
    if key == ord('s') or key == ord('ы'):
        cv2.imwrite('photo ' + str(PhotoCntr) + '.png', frame)
        PhotoCntr += 1

cv2.destroyAllWindows()
vs.stop()

3
10
10
11
10
10
11
11
11
10
10
10
11
11
10
10
10
10
9
9
9
8
8
7
7
8
6
5
5
6
7
7
7
7
6
8
8
8
7
7
7
7
6
6
6
6
8
7
8
7
9
9
4
5
8
9
9
9
8
9
10
10
9
10
10
10
10
10
9
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
7
8
8
8
8
8
8
8
8
8
8
8
7
8
8
8
8
7
8
